In [21]:
from datetime import datetime as dt
from IPython.display import display, HTML
import ipywidgets as widgets
import urllib
import pandas as pd

In [22]:
collection_input=['conservation-area', 'article-4-direction-area', 'article-4-direction', 'listed-building-outline', 'tree', 'tree-preservation-order', 'tree-preservation-zone']
organisation_input=''
severity_input=[] # list of severities e.g ["error", "warning", "info", "notice"]
line_number_input=''

datasette_url = "https://datasette.planning.data.gov.uk/"

current_date = dt.now().date()

date_query = f" where SUBSTRING(s.entry_date, 1, 4) = '2023'"


collection_dfs=[]
output_collection_names=[]
for collection in collection_input:
    query = ""
    if collection_input:
        query = f" and s.collection = '{collection}'"

    if organisation_input:
        query = query + f" and s.organisation = '{organisation_input}'"

    params = urllib.parse.urlencode({
        "sql": f"""
        select re.resource, re.endpoint, s.organisation, s.entry_date, e.endpoint_url
        from resource_endpoint re
        inner join endpoint e
        on re.endpoint = e.endpoint
        inner join source s
        on e.endpoint = s.endpoint
        {date_query}
        {query}

        """,
        "_size": "max"
    })

    url = f"{datasette_url}digital-land.csv?{params}"
    df = pd.read_csv(url)
    if (df.empty):
        print(print("\033[1m No results found for ", collection))
    else:
        collection_dfs.append(df)
        output_collection_names.append(collection)

 No results found for  listed-building-outline
None
 No results found for  tree
None
 No results found for  tree-preservation-order
None
 No results found for  tree-preservation-zone
None


,resource,endpoint,organisation,entry_date,endpoint_url
0,2c62d64f790d690488b5c523ac1f0b3080848708c767b938e78058a5c0d0ae02,3359f0dcc3566387183cdf903722f1771dde9cba1f16c78d9afd1c13d6589007,local-authority-eng:CMD,2023-08-14T09:09:18Z,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD
1,7306c46664b948b9d0ded55f661aa150734829d0c2d924c26dffd6f4328b133c,3359f0dcc3566387183cdf903722f1771dde9cba1f16c78d9afd1c13d6589007,local-authority-eng:CMD,2023-08-14T09:09:18Z,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD
2,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed917a7fe0280aae9b4fa,ecf6d0fbf0312df98a2f7e8e8b41a65757a5f1eb3ad31859a55c68c6a2300212,local-authority-eng:SWK,2023-08-15T13:13:41Z,https://www.southwark.gov.uk/assets/attach/194104/Conservation-Areas.gpkg


In [23]:
params = urllib.parse.urlencode({
    "sql": f"""
    select description, issue_type, severity
    from issue_type
    """,
    "_size": "max"
})

url = f"{datasette_url}digital-land.csv?{params}"
issue_type = pd.read_csv(url)

In [24]:
issues_dfs=[]
for idx, collection_df in enumerate(collection_dfs):
    query=""
    if line_number_input:
        query = f" and line_number = '{line_number_input}'"

    resources = collection_df['resource'].tolist()
    issues = []
    for resource in resources:
        params = urllib.parse.urlencode({
        "sql": f"""
        select field,issue_type,dataset,resource,value, line_number
        from issue
        where resource = '{resource}'
        {query}
        """,
        "_size": "max"
        })
        url = f"{datasette_url}{collection_input[idx]}.csv?{params}"
        df1 = pd.read_csv(url)
        issues.append(df1)
    df1 = pd.concat(issues, ignore_index=True)
    issues_with_type = df1.merge(issue_type, left_on='issue_type', right_on='issue_type')
    issues_dfs.append(issues_with_type)

In [25]:
output_dfs=[]
for idx, collection_df in enumerate(collection_dfs):
    collection_issues_df = collection_df.merge(issues_dfs[idx], left_on='resource', right_on='resource')
    collection_issues_df = collection_issues_df.reindex(columns=['organisation', 'dataset', 'resource', 'endpoint_url', 'entry_date', 'field', 'line_number', 'issue_type', 'value', 'severity', 'description']).set_index('organisation')
    if (severity_input):
        collection_issues_df = collection_issues_df.loc[collection_issues_df['severity'].isin(severity_input)]
    output_dfs.append(collection_issues_df)

In [26]:
def f(dataset):
    index = output_collection_names.index(dataset)
    display(HTML(output_dfs[index].to_html()))

selector = widgets.RadioButtons(
    options=output_collection_names,
    description='Select dataset to display',
    disabled=False
)
ui = widgets.VBox([selector])
out = widgets.interactive_output(f, {'dataset': selector})
display(ui, out)

Output()